In [2]:
import osmnx as ox
import folium
import random
import pandas as pd

# ---------- 1) Download Zurich's street network ----------
place = "Zürich, Switzerland"
G = ox.graph_from_place(place, network_type="walk")   # pedestrian network
G_bike = ox.graph_from_place(place, network_type="bike")  # bicycle network

# ---------- 2) Function to generate random points on the graph ----------
def random_points_on_graph(G, n_points):
    edges = list(G.edges(keys=True, data=True))
    points = []
    for _ in range(n_points):
        u, v, key, data = random.choice(edges)
        geom = data.get("geometry", None)
        if geom:
            point = geom.interpolate(random.random(), normalized=True)
            points.append((point.y, point.x))
        else:
            # If the edge has no geometry, interpolate directly between the nodes
            lat1, lon1 = G.nodes[u]["y"], G.nodes[u]["x"]
            lat2, lon2 = G.nodes[v]["y"], G.nodes[v]["x"]
            lat = lat1 + random.random() * (lat2 - lat1)
            lon = lon1 + random.random() * (lon2 - lon1)
            points.append((lat, lon))
    return points

# ---------- 3) Generate pedestrian and bicycle points ----------
ped_points = random_points_on_graph(G, 500)
bike_points = random_points_on_graph(G_bike, 500)

df_ped = pd.DataFrame(ped_points, columns=["lat", "lon"])
df_ped["type"] = "pedestrian"

df_bike = pd.DataFrame(bike_points, columns=["lat", "lon"])
df_bike["type"] = "bicycle"

df_all = pd.concat([df_ped, df_bike], ignore_index=True)

# ---------- 4) Create interactive map ----------
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13, tiles="CartoDB dark_matter")
color_map = {"pedestrian": "#FF4D6D", "bicycle": "#00FFFF"}

for _, row in df_all.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=2,
        color=color_map[row["type"]],
        fill=True,
        fill_opacity=0.8
    ).add_to(m)

# ---------- 5) Save map ----------
output_html = "Zurich_street_based_pedestrians_bicycles.html"
m.save(output_html)
print("✅ Map saved:", output_html)


✅ Map saved: Zurich_street_based_pedestrians_bicycles.html
